In [1]:
import skmob
from skmob.models.sts_epr import STS_epr
from skmob.models.epr import Ditras
import pandas as pd
import dill
import pickle
from skmob.preprocessing import clustering
from skmob.models.epr import Ditras
from skmob.models.markov_diary_generator import MarkovDiaryGenerator

In [2]:
df_real = pd.read_csv("./NewYork_trajectories_final.csv", sep="\t")

tdf_real = skmob.TrajDataFrame(df_real, latitude='lat', longitude='lon',
                                               user_id='uid', datetime='datetime')

tdf_real[:5]

ctdf_real = clustering.cluster(tdf_real)

start_time = pd.to_datetime('2019/01/01 08:00:00')

In [3]:
mdg = MarkovDiaryGenerator()
mdg.fit(ctdf_real, 10, lid='cluster')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.58it/s]


### TEST with SEED

In [4]:
x = mdg.generate(100, start_time, random_state=42)

for i in range(1999):
    y = mdg.generate(100, start_time, random_state=42)
    assert x.equals(y)

### FIX THE REPRODUCIBILITY of the models

In [5]:
start = pd.to_datetime('2012/04/10 00:00:00')
end = pd.to_datetime('2012/04/20 00:00:00')

social_graph_filename = './NewYork_social_graph_final.csv'
diary_filename = './diary_NewYork.pickle'

#load the social graph and build an edge list representation
edges = []
social_graph_real = pd.read_csv(social_graph_filename, sep='\t')
social_graph_real.apply(lambda row: edges.append((row['Source'],row['Target'])),axis=1);
social_graph = edges

#load the Mobility Diary Generator
diary = dill.load(open(diary_filename, 'rb'))

tex = pickle.load(open('./tex_NewYork_sq_300.pickle','rb'))
tex = tex[:50]

### STS-EPR

#### seed correction

In [6]:
sts_epr = STS_epr()
rand_seed = 42
no_seed_1 = sts_epr.generate(start, end, social_graph=social_graph,
                                        spatial_tessellation=tex, rsl = True, 
                            diary_generator=diary,  relevance_column='relevance',
                                        random_state=rand_seed, show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:07<00:00, 31.53it/s]


In [7]:
sts_epr = STS_epr()
rand_seed = 42
no_seed_2 = sts_epr.generate(start, end, social_graph=social_graph,
                                        spatial_tessellation=tex, rsl = True, 
                            diary_generator=diary,  relevance_column='relevance',
                                        random_state=rand_seed, show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████| 240/240 [00:07<00:00, 32.82it/s]


In [8]:
no_seed_1.equals(no_seed_2)

True

### DITRAS

In [9]:
ditras = Ditras(diary)
ditras_no_seed_1 = ditras.generate(start, end, tex, n_agents=100, show_progress=True,
                         relevance_column='relevance',random_state=rand_seed)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\giuli\Documents\GitHub\scikit-mobility\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 206.02it/s]


In [10]:
ditras = Ditras(diary)
ditras_no_seed_2 = ditras.generate(start, end, tex, n_agents=100, show_progress=True,
                         relevance_column='relevance',random_state=rand_seed)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\giuli\Documents\GitHub\scikit-mobility\skmob\models\gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 201.83it/s]


In [11]:
ditras_no_seed_1.equals(ditras_no_seed_2)

True